In [12]:
import sys
sys.version

'3.10.8 (main, Nov 24 2022, 08:09:04) [Clang 14.0.6 ]'

In [13]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.xkcd()

import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
import statsmodels.api as sm
import math

pd.options.mode.chained_assignment = None 

In [14]:
from functions import *

In [16]:
def filter_by_sector(tickers_list_,new_ticker_list,zacks_sector_code_list,tickers_remove_list,start_date,end_date):

    for ticker in tickers_list_:

        if ticker in tickers_remove_list:
            continue

        df_fc = quandl.get_table('ZACKS/FC',ticker = ticker, per_end_date = {'gte':start_date,'lte':end_date},qopts={"columns":"zacks_sector_code"})

        if len(df_fc)!=0:
            sector_code_list = list(df_fc['zacks_sector_code'])

            if bool(set(zacks_sector_code_list) & set(sector_code_list))==False:
                new_ticker_list.append(ticker)

    return new_ticker_list

In [17]:
start_date = '2014-06-30'
end_date = '2022-06-30'

df_tickers = pd.read_csv("zacks-tickers.csv")
tickers_list = list(df_tickers['3262Q'])

zacks_sector_code_list = [5,13]
tickers_remove_list = ['ABEO']

new_ticker_list = []
new_ticker_list = filter_by_sector(tickers_list[0:1000],new_ticker_list,zacks_sector_code_list,tickers_remove_list,start_date,end_date)

# new_ticker_list

In [18]:
df_new_ticker1 = pd.DataFrame(new_ticker_list, columns =['ticker'])

In [19]:
df_new_ticker1
    

,ticker
0,A
1,AA
2,AAAP
3,AAC
4,AACAY
...,...
577,BIG
578,BIGG
579,BIIB
580,BINDQ


In [20]:
df_new_ticker1.to_csv('filter_by_sector_1000.csv',index=False)

In [21]:
new_ticker_list5 = []
new_ticker_list5 = filter_by_sector(tickers_list[4000:5000],new_ticker_list,zacks_sector_code_list,tickers_remove_list,start_date,end_date)

df_new_ticker5 = pd.DataFrame(new_ticker_list, columns =['ticker'])
df_new_ticker5.to_csv('filter_by_sector_5000.csv',index=False)

In [22]:
len(df_new_ticker5)

1192

In [23]:
df_new_ticker5

,ticker
0,A
1,AA
2,AAAP
3,AAC
4,AACAY
...,...
1187,MCRS
1188,MCS
1189,MCUJF
1190,MCUR


In [24]:
len(tickers_list)

8912

In [9]:
def filter_by_adj_close(tickers_list_,start_date,end_date,num):

    filter_by_adj_close_list = []

    for ticker in tickers_list_:
        df1 = get_adj_close(ticker,start_date,end_date)
        if len(df1) == num:
            filter_by_adj_close_list.append(ticker)

    return filter_by_adj_close_list


start_date = '2014-06-30'
end_date = '2022-06-30'

df5 = pd.read_csv("filter_by_sector_5000.csv")
ticker_list5 = list(df5['ticker'])
print(len(ticker_list5))

filter_by_adj_close_list5 = filter_by_adj_close(ticker_list5,start_date,end_date,2016)

1192


In [10]:
len(filter_by_adj_close_list5)

467

In [11]:
df_new_ticker5 = pd.DataFrame(filter_by_adj_close_list5, columns =['ticker'])
df_new_ticker5.to_csv('filter_by_adj_close_5000.csv',index=False)

In [18]:
def filter_by_debt_ratio(df_ticker,start_date,end_date):

    df_ticker['debt_ratio_greater_than_0.1'] = 0

    for i in range(len(df_ticker)):

        ticker = df_ticker['ticker'][i]

        df_zack = generate_zacks_data(ticker,start_date,end_date)
        df_price = get_adj_close(ticker,start_date,end_date)

        df = df_price.join(df_zack,how='outer')
        df['adj_close'] =  df['adj_close'].fillna(method='ffill')

        df1 = debt_to_market_cap(df,df_zack)

        count = df1[['debt_to_market_cap']][df1[['debt_to_market_cap']]  > 0.1].count()

        df_ticker['debt_ratio_greater_than_0.1'][i] = count

        # if i==100:
        #     break

    return df_ticker

In [19]:
df_ticker_adj_close = pd.read_csv("filter_by_adj_close.csv")
print(len(df_ticker_adj_close))

df_ticker_adj_close = df_ticker_adj_close[0:500]
df_ticker_adj_close.reset_index(inplace=True)
print(len(df_ticker_adj_close))

df_ticker_debt_ratio1 = filter_by_debt_ratio(df_ticker_adj_close,start_date,end_date)

1975
500


In [21]:
df_ticker_debt_ratio1.to_csv('filter_by_debt_ratio_1.csv',index=False)

In [22]:
df_ticker_debt_ratio1

,index,ticker,debt_ratio_greater_than_0.1
0,0,A,1983
1,1,AAL,983
2,2,AAOI,1835
3,3,AAON,86
4,4,AAP,1988
...,...,...,...
495,495,BRKR,1996
496,496,BSET,5
497,497,BSQR,423
498,498,BSX,1998
